## 先生成核矩阵保存起来

In [139]:
from shogun_tools import *
from shogun import GMNPSVM, CSVFile

def combined_kernel(file_type, data_name, operate_type):
    if file_type == '4':
        X, y = loadFromMat(data_name)
    elif file_type == '5':
        X, y = loadFromLibsvm(data_name)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)  
    if type(X_train)==scipy.sparse.csr.csr_matrix and type(X_test)==scipy.sparse.csr.csr_matrix:
        X_train=X_train.todense()
        X_test=X_test.todense()
    X_train=X_train.T
    X_test=X_test.T
    y_train=y_train.reshape(y_train.size,).astype('float64')
    y_test=y_test.reshape(y_test.size,).astype('float64')

    kernel = CombinedKernel()
    feats_train = CombinedFeatures()
    feats_test = CombinedFeatures()
    subkfeats_train = RealFeatures(X_train)
    subkfeats_test = RealFeatures(X_test)
    for i in range(-10,11):
        subkernel = GaussianKernel(pow(2,i+1))
        feats_train.append_feature_obj(subkfeats_train)
        feats_test.append_feature_obj(subkfeats_test)
        kernel.append_kernel(subkernel)
    kernel.init(feats_train, feats_train)
    tmp_train_csv = NamedTemporaryFile(suffix=data_name+'_combined.csv')
    if operate_type=='save':        
        km_train=kernel.get_kernel_matrix()
        f=CSVFile(tmp_train_csv.name, "w")
        kernel.save(f)
    elif operate_type=='load':
        f=CSVFile(tmp_train_csv.name, "r")
        kernel.load(f)
        
    labels=MulticlassLabels(y_train)

    mkl = MKLMulticlass(C, kernel, labels)

    mkl.set_epsilon(epsilon)
    mkl.parallel.set_num_threads(num_threads)
    mkl.set_mkl_epsilon(mkl_epsilon)
    mkl.set_mkl_norm(mkl_norm)

    mkl.train()
    kernel.init(feats_train, feats_test)
    out =  mkl.apply().get_labels()
    print out.shape
    print out, y_test
    print sum(out==y_test)/float(len(out))


In [149]:
times=30
C=100 #[1, 10, 100, 1000]    
epsilon=1e-5
mkl_epsilon=0.001
test_size=0.2
width=8
num_threads=32
mkl_norm=1

import time
start = time.time()
combined_kernel('4', 'plant','load')
end=time.time()
print 'use time : ' + str(end-start)



(188,)
[ 0.  3.  0.  2.  2.  3.  3.  1.  2.  1.  1.  2.  2.  1.  1.  2.  1.  2.
  0.  0.  1.  2.  2.  3.  1.  3.  1.  3.  1.  1.  1.  1.  1.  0.  1.  1.
  2.  2.  2.  0.  2.  1.  1.  3.  1.  2.  3.  1.  2.  3.  2.  2.  2.  2.
  1.  1.  1.  3.  1.  2.  1.  2.  1.  2.  1.  1.  1.  1.  2.  2.  2.  1.
  2.  2.  1.  2.  0.  3.  2.  1.  1.  2.  3.  1.  2.  1.  3.  3.  3.  0.
  1.  0.  3.  2.  3.  1.  1.  1.  3.  1.  2.  1.  0.  2.  1.  3.  1.  2.
  1.  1.  2.  1.  2.  2.  3.  1.  2.  1.  2.  3.  2.  1.  2.  3.  2.  1.
  0.  1.  2.  0.  1.  2.  2.  2.  1.  3.  0.  1.  2.  1.  0.  2.  2.  2.
  3.  1.  1.  2.  1.  1.  1.  3.  2.  3.  1.  2.  1.  1.  1.  1.  2.  1.
  1.  1.  2.  1.  2.  1.  3.  1.  2.  2.  1.  0.  3.  3.  2.  2.  2.  2.
  1.  2.  1.  1.  2.  1.  1.  3.] [ 0.  3.  0.  3.  2.  3.  3.  1.  2.  1.  1.  3.  2.  1.  1.  2.  1.  2.
  1.  1.  1.  3.  2.  3.  1.  3.  1.  0.  1.  1.  1.  1.  1.  0.  3.  1.
  2.  2.  2.  3.  2.  1.  1.  3.  1.  2.  3.  1.  2.  3.  0.  2.  2.  2.
  0.  1.  

In [33]:
k2.get_kernel_matrix()[1,0]

8.2535074399873327

In [28]:
k2.get_kernel_matrix().T==k1.get_kernel_matrix()

array([[ True, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ..., 
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False,  True]], dtype=bool)

In [32]:
k1.get_kernel_matrix()[0,1]

8.1576132247494435

In [ ]:
\begin{table*}[t]
\small
\footnotesize
%\scriptsize
%\tiny
%\renewcommand{\captionfont}{\small}
   \caption{
    \small Comparison of average test accuracies of our \texttt{Conv-MKL} and \texttt{SMSD-MKL} with the others including
    the linear multi-class SVM (LSVM),
    One-against-One (One vs. One),
    One-against-the-Rest(One vs. Rest),
    Generalized Minimal Norm Problem solver (GMNP),
    the $\ell_1$-norm Multiclass MKL ($\ell_1$ MC-MKL),
    the $\ell_2$-norm Multiclass MKL ($\ell_2$ MC-MKL),
    and mixed-norm MKL solved by stochastic gradient descent (UFO-MKL).
    We bold the numbers of the best method, and underline the numbers of the other methods
    which are not significantly worse than the best one.
   %We set the numbers of our method (EPSVM) to be bold if our method outperforms all other methods (KTA, CKTA, FSM, 3-CV, 5-CV and 10-CV).
   }
   \label{tabel:accuracy}
    %\centering
    \begin{tabular*}{\linewidth}{@{\extracolsep{-0.25cm}}lccccccccc}
    \toprule
                   & Conv-MKL                  & SMSD-MKL                 & C\&S               & One vs. One              & One vs. Rest                                              & GMNP                      & $\ell_1$ MC-MKL          & $\ell_2$ MC-MKL    & UFO-MKL                  \\ \hline
plant              &77.14$\pm$2.25             & \textbf{78.01$\pm$2.17}  & 70.12$\pm$2.96     & 75.83$\pm$2.69           &75.17$\pm$2.68       &75.42$\pm$3.64    & \underline{77.60$\pm$2.63}&75.49$\pm$2.48            &76.77$\pm$2.42\\
psortPos           &74.41$\pm$3.35             & \textbf{76.23$\pm$3.39}  &63.85$\pm$3.94      &73.33$\pm$4.21            &71.70$\pm$4.89       & 73.55$\pm$4.22   &71.87$\pm$4.87             &70.70$\pm$4.89            &74.56$\pm$4.04\\
psortNeg           &74.07$\pm$2.16             & \textbf{74.66$\pm$1.90}  &57.85$\pm$2.49      &73.74$\pm$2.87            &71.94$\pm$2.50   &\underline{74.27$\pm$2.51}   &72.83$\pm$2.20                             &72.42$\pm$2.65      &73.80$\pm$2.26 \\
nonpl              & \textbf{79.15$\pm$1.51}   &78.69$\pm$1.58            &75.16$\pm$1.48      &77.78$\pm$1.52            &77.49$\pm$1.53 &78.35$\pm$1.46    &77.89$\pm$1.79             &77.95$\pm$1.64            &78.07$\pm$1.56\\
sector             &\underline{92.83$\pm$2.62} & \textbf{93.39$\pm$0.70}  &93.16$\pm$0.66      &90.61$\pm$0.69            &\underline{93.34$\pm$0.61}                    &\                         &\                   &92.15$\pm$2.57            &92.60$\pm$0.47\\
segment            &96.79$\pm$0.91             & \textbf{97.62$\pm$0.83}  &95.07$\pm$1.11      &97.08$\pm$0.61            &97.02$\pm$0.80       &96.87$\pm$0.80    &96.98$\pm$0.64             &\underline{97.58$\pm$0.68}&97.20$\pm$0.82\\
vehicle            &79.35$\pm$2.27             & 77.28$\pm$2.78           &75.61$\pm$3.56      &82.72$\pm$1.92            &\textbf{85.11$\pm$1.94}       &81.57$\pm$2.24    &74.96$\pm$2.93             &76.27$\pm$3.15            &76.92$\pm$2.83\\
vowel              &98.82$\pm$1.19             &\textbf{98.83$\pm$5.57}   &62.32$\pm$4.97      &98.12$\pm$1.76            &98.22$\pm$1.83       &97.04$\pm$1.85    &98.27$\pm$1.22             &97.86$\pm$1.75            &98.22$\pm$1.62\\
wine               &\textbf{99.63$\pm$0.96}    &\textbf{99.63$\pm$0.96}   &97.87$\pm$2.80      &97.24$\pm$3.05            &98.14$\pm$3.04       &97.69$\pm$2.43    &98.61$\pm$1.75             &98.52$\pm$1.89            &99.44$\pm$1.13            \\
dna                &96.08$\pm$0.83             &\textbf{96.30$\pm$0.79}   &92.02$\pm$1.50      &95.89$\pm$0.56            &95.61$\pm$0.73       &94.60$\pm$0.94    &\underline{96.27$\pm$0.68}             &95.06$\pm$0.92            &95.84$\pm$0.61\\
glass              &\textbf{75.19$\pm$5.05}    & 73.72$\pm$5.80           &63.95$\pm$6.04      &71.98$\pm$5.75            &70.00$\pm$5.75       &71.24$\pm$8.14    &69.07$\pm$8.08             &74.03$\pm$6.41            &72.46$\pm$6.12\\
iris               &\textbf{98.00$\pm$2.25}    &\underline{97.33$\pm$2.53}&89.67$\pm$8.71      &96.76$\pm$2.63            &96.11$\pm$3.04       &96.11$\pm$4.11    &96.00$\pm$3.65             &95.44$\pm$3.66            &95.56$\pm$3.07\\
svmguide2          &82.69$\pm$5.65             &\textbf{85.17$\pm$3.83}   &81.10$\pm$4.15      &\underline{84.79$\pm$3.45}&\underline{84.27$\pm$3.03}  &81.77$\pm$3.45    &83.16$\pm$3.63             &\underline{83.84$\pm$4.21}            &82.91$\pm$3.09\\
satimage           &91.64$\pm$0.88             &\underline{91.78$\pm$0.82}&84.95$\pm$1.15      &90.67$\pm$0.91            &89.29$\pm$0.96       &89.97$\pm$0.81    &\underline{91.86$\pm$0.62} &90.43$\pm$1.27            &\textbf{91.92$\pm$0.83}\\
\bottomrule
\end{tabular*}
\vspace{-0.3cm}
\end{table*}